# 【Q13】What is the percentage of female researcher in your group ?

- 【Q13】What is the percentage of female researcher in your group ?
- 【Q14】What do you think about the percentage above ?

In [1]:
import altair as alt
import pandas as pd
import scipy as sp
import titanite as ti

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciPy {sp.__version__}")
print(f"Titanite {ti.__version__}")

Altair 5.0.1
Pandas 2.0.3
SciPy 1.11.1
Titanite 0.4.1


データを読み込む

In [2]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
data = d.read()
# data.info()

2023-07-23 22:56:58.400 | INFO     | titanite.preprocess:categorical_data:123 - Categorize
2023-07-23 22:56:58.409 | INFO     | titanite.preprocess:binned_data:229 - Binned


In [3]:
# x = ["q13", "q13_binned"]
#x = ["q13_binned", "q14"]
x = ["q14"]
headers = d.crosstab_headers(x, list(data.columns))

In [4]:
cross_tabs, heatmaps, chi2_data = ti.core.crosstab_loop(data, headers)
chi2_data

,questions,p_value,statistic,dof
0,q14-q01,2.307692e-01,46.222813,40
1,q14-q02,3.287573e-01,22.218793,20
2,q14-q05,3.350292e-01,32.715313,30
3,q14-q06,4.458944e-01,40.552624,40
4,q14-q07,8.266793e-03,31.196946,15
5,q14-q08,2.559302e-01,29.192963,25
6,q14-q09,1.941626e-02,21.249990,10
7,q14-q11,4.317236e-01,10.100468,10
8,q14-q12_genderbalance,1.667273e-40,257.645141,25
9,q14-q12_diversity,3.148340e-16,130.378606,25


In [5]:
mark = alt.Chart(chi2_data).mark_point().encode(
    alt.X("p_value"),
    alt.Y("questions"),
    tooltip=["questions", "p_value", "dof", "statistic"]
)

rule = (
    alt.Chart()
    .mark_rule(color="orange")
    .encode(x=alt.datum(0.05))
)

(mark + rule).properties(
    width=500,
)

alt.LayerChart(...)

``p < 0.05``の項目を調べる

In [6]:
p005 = chi2_data.query("p_value < 0.05")
p005

,questions,p_value,statistic,dof
4,q14-q07,8.266793e-03,31.196946,15
6,q14-q09,1.941626e-02,21.249990,10
8,q14-q12_genderbalance,1.667273e-40,257.645141,25
9,q14-q12_diversity,3.148340e-16,130.378606,25
10,q14-q12_equity,1.521236e-10,97.708775,25
11,q14-q12_inclusion,6.152220e-12,105.940428,25
12,q14-q17_genderbalance,1.573677e-15,104.830489,15
13,q14-q17_diversity,4.070271e-12,86.703262,15
14,q14-q17_equity,1.445257e-06,55.546239,15
15,q14-q17_inclusion,8.831318e-05,44.606799,15


In [7]:
for name, hm in heatmaps.items():
    if name in list(p005.questions):
        hm.properties(
            title=name,
            width=400,
            height=400,
        ).display()

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

...

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)